<h1>Generation Prompt from Process</h1>

In [ ]:
from pathlib import Path
import pandas as pd
import re

# 템플릿 파일 로드 (utf-8 기준)
with open("FastMCP_Code_Generation_Prompt_EN.txt", encoding="utf-8") as f:
    template = f.read()

def sanitize_filename(name: str) -> str:
    """파일명으로 사용 가능한 안전한 이름으로 변환"""
    name = re.sub(r"[^\w\s-]", "", name)  # 한글 포함 허용 시 제거
    return name.replace(" ", "_").replace("/", "_")

def generate_prompt_from_template(template: str, process_group: str, idshorts: list[str]) -> str:
    idshort_lines = "\n  - " + "\n  - ".join(idshorts)
    prompt = template.replace("- Process Group: milling_3_axis_v1", f"- Process Group: {process_group}")
    prompt = prompt.replace(
        """- spindleSpeedControl
  - toolChangeRequest
  - axisMovement
  - coolantFlowToggle
  - emergencyStopSignal""",
        idshort_lines
    )
    return prompt

def generate_prompts(csv_path: str, output_dir: str = "prompts_en"):
    df = pd.read_csv(csv_path, encoding="utf-8")
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    for _, row in df.iterrows():
        pg = row["Process Group"]
        ids = [s.strip() for s in str(row["Common idShorts"]).split(",") if s.strip()]
        content = generate_prompt_from_template(template, pg, ids)
        filename = sanitize_filename(pg) + ".txt"
        prompt_path = Path(output_dir) / filename
        prompt_path.write_text(content, encoding="utf-8")
        print(f"✅ Prompt generated: {prompt_path}")

# 사용 예시
if __name__ == "__main__":
    generate_prompts("valid_final.csv")


<h1>Local Gemma3 Load Test</h1>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def test_gemma_model_load(model_path: str = "/home/h100-ku/gemma-3-27b-it"):
    try:
        print("✅ Load Tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        print("✅ Load Model...")
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )
        print("✅ Success!")
    except Exception as e:
        print("❌ Fail", e)

# 테스트 실행
test_gemma_model_load()


<h1>Auto Code Generation Public AI</h1>

In [ ]:
import os
from pathlib import Path
import re

# API 키 불러오기
from dotenv import load_dotenv
load_dotenv()

import openai
import anthropic
import google.generativeai as genai

# ── 유틸 함수 ───────────────────────────
def clean_code_block(text: str) -> str:
    code = re.sub(r"^```(?:python)?\n?", "", text.strip())
    return re.sub(r"\n?```$", "", code)

def save_code(code_text: str, model: str, process_group: str) -> str:
    filename = f"generated_{model}_{process_group}.py"
    Path(filename).write_text(clean_code_block(code_text), encoding="utf-8")
    return filename

# ── GPT (OpenAI) ─────────────────────────
def generate_from_gpt(prompt_file_path: str, process_group: str) -> str:
    with open(prompt_file_path, encoding="utf-8") as f:
        prompt = f.read()

    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("❌ OPENAI_API_KEY not set in .env file or environment variables.")
    openai.api_key = api_key

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            temperature=0.3,
            messages=[{"role": "user", "content": prompt}]
        )
        code = response.choices[0].message.content
        return save_code(code, "gpt4", process_group)
    except Exception as e:
        print("❌ Error GPT API:", e)
        raise

# ── Claude (Anthropic) ───────────────────
def generate_from_claude(prompt_file_path: str, process_group: str) -> str:
    with open(prompt_file_path, encoding="utf-8") as f:
        prompt = f.read()

    api_key = os.getenv("ANTHROPIC_API_KEY")
    if not api_key:
        raise ValueError("❌ ANTHROPIC_API_KEY not set in .env file or environment variables.")
    client = anthropic.Anthropic(api_key=api_key)

    try:
        response = client.messages.create(
            model="claude-3-7-sonnet-20250219",
            max_tokens=4096,
            temperature=0.3,
            messages=[{"role": "user", "content": prompt}]
        )
        code = response.content[0].text
        return save_code(code, "claude", process_group)
    except Exception as e:
        print("❌ Error Claude API:", e)
        raise

# ── Gemini (Google) ──────────────────────
def generate_from_gemini(prompt_file_path: str, process_group: str) -> str:
    with open(prompt_file_path, encoding="utf-8") as f:
        prompt = f.read()

    api_key = os.getenv("GOOGLE_API_KEY")
    if not api_key:
        raise ValueError("❌ GOOGLE_API_KEY not set in .env file or environment variables.")
    genai.configure(api_key=api_key)

    try:
        model = genai.GenerativeModel("gemini-2.5-pro-preview-05-06")
        response = model.generate_content(prompt)
        code = response.text
        return save_code(code, "gemini", process_group)
    except Exception as e:
        print("❌ Error Gemini API:", e)
        raise


<h1>Auto Code Generation Local AI</h1>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from pathlib import Path
import re

def clean_code_block(text: str) -> str:
    code = re.sub(r"^```(?:python)?\n?", "", text.strip())
    return re.sub(r"\n?```$", "", code)

def save_code(code_text: str, model: str, process_group: str) -> str:
    filename = f"generated_{model}_{process_group}.py"
    Path(filename).write_text(clean_code_block(code_text), encoding="utf-8")
    return filename

def load_gemma_model(model_path: str = "/home/h100-ku/gemma-3-27b-it"):
    print("🔁 Gemma 모델 로딩 중...")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        device_map="auto"
    )
    print("✅ Gemma 모델 로드 완료")
    return tokenizer, model

def generate_with_loaded_gemma(prompt_file_path: str, process_group: str, tokenizer, model) -> str:
    try:
        with open(prompt_file_path, encoding="utf-8") as f:
            prompt = f.read()

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=8192,
                temperature=0.3,
                do_sample=True,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id
            )

        code = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return save_code(code, "gemma", process_group)

    except Exception as e:
        print("❌ Gemma 생성 실패:", e)
        raise


In [ ]:
process_group = ["PressPress_Servo_Type","Heating_Heating_Quenching","Rolling_Rolling_hot","Heating_Heating__Aging","AFPMMotorProductionType"]
prompt_path = ["prompts_en/PressPress_Servo_Type.txt","prompts_en/Heating_Heating_Quenching.txt","prompts_en/Rolling_Rolling_hot.txt","prompts_en/Heating_Heating_Aging.txt","prompts_en/AFPMMotorProductionType.txt"]

for a in range(len(process_group)):
    # 모델별 코드 생성 및 저장
    print("request GPT")
    gpt_path = generate_from_gpt(prompt_path[a], process_group[a])
    print("request GPT Complete")
    print("request Claude")
    claude_path = generate_from_claude(prompt_path[a], process_group[a])
    print("request Claude Complete")
    print("request Gemini")
    gemini_path = generate_from_gemini(prompt_path[a], process_group[a])
    print("request Gemini Complete")

print("✅ GPT code saved:", gpt_path)
print("✅ Claude code saved:", claude_path)
print("✅ Gemini code saved:", gemini_path)

<h1>Evaluation Code</h1>

In [ ]:
import os
from pathlib import Path
import anthropic
import openai
from dotenv import load_dotenv

load_dotenv()

# ───── generate Evaluation prompt ─────
def build_eval_prompt(code_text: str, process_group: str) -> str:
    prompt_path = Path(f"prompts_evaluate/{process_group}.txt")
    if not prompt_path.exists():
        raise FileNotFoundError(f"프롬프트 파일 없음: {prompt_path}")
    with open(prompt_path, encoding="utf-8") as f:
        prompt_template = f.read()
    return prompt_template.replace("{code}", code_text.strip())


#-----------Evaluate With Claude4 ---------------------------
def evaluate_with_claude(code_path: str, process_group: str) -> str:
    client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
    code = Path(code_path).read_text(encoding="utf-8")
    prompt = build_eval_prompt(code, process_group)
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=8196,
        temperature=0.2,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text
#-----------Evaluate With O3 ---------------------------
def evaluate_with_gpt_o3(code_path: str, process_group: str) -> str:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    code = Path(code_path).read_text(encoding="utf-8")
    prompt = build_eval_prompt(code, process_group)
    response = openai.ChatCompletion.create(
        model="o3-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

# ───── Start Evaluate Function ─────
def evaluate_all_for_process(process_group: str) -> dict:
    code_models = ["gpt4", "claude", "gemini","gemma"]  # 코드 작성 모델
    eval_models = ["gpt4", "claude"]            # 평가 모델 (o3-mini, claude)

    results = {}

    for code_model in code_models:
        filename = f"generated_{code_model}_{process_group}.py"
        print(f"\n=== evaluation Code: {filename} ===")

        if not os.path.exists(filename):
            print("⚠️ No Code File.")
            results[code_model] = "No Code File"
            continue

        evaluations = {}

        for eval_model in eval_models:
            try:
                if eval_model == "claude":
                    eval_result = evaluate_with_claude(filename, process_group)
                elif eval_model == "gpt4":
                    eval_result = evaluate_with_gpt_o3(filename, process_group)
                else:
                    eval_result = "❌ Unsupported Model"

                evaluations[eval_model] = eval_result

            except Exception as e:
                print(f"❌ {eval_model.upper()} evaluate Exception:", e)
                evaluations[eval_model] = f"evaluate Exception: {e}"

        # save evaluate Result
        result_file = Path(f"results/{process_group}_{code_model}_eval.txt")
        result_file.parent.mkdir(exist_ok=True)

        # 📄 add target Fila Name
        header = f"📄 Evaluate target: {filename}\n"
        content = "\n".join([
            f"\n===== {eval.upper()} result =====\n{res.strip()}"
            for eval, res in evaluations.items()
        ])
        result_file.write_text(header + content, encoding="utf-8")


        results[code_model] = evaluations

    return results


if __name__ == "__main__":
    process_groups = [
        "PressPress_Servo_Type",
        "Heating_Heating_Quenching",
        "Rolling_Rolling_hot",
        "Heating_Heating__Aging",
        "AFPMMotorProductionType"
    ]
    for pg in process_groups:
        print(f"\n📂 Start Evaluate: {pg}")
        evaluate_all_for_process(pg)


In [ ]:
process_groups = ["PressPress_Servo_Type",
        "Heating_Heating_Quenching",
        "Rolling_Rolling_hot",
        "Heating_Heating__Aging",
        "AFPMMotorProductionType"]
for pg in process_groups:
    print(f"\n📂 공정 평가 시작: {pg}")
    evaluate_all_for_process(pg)

<h1>Heat Map</h1>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# === 1. Input Result Data ===
data = {
    "Task": [],
    "Model": [],
    "o3": [],
    "Claude 4": []
}

tasks = {
    "AFPMMotorProductionType": [
        ("GPT 4o", 54, 47),
        ("Claude 3.7 Sonnet", 67, 45),
        ("Gemini 2.5 pro (preview)", 86, 75),
        ("Gemma3 27B", 92, 92)
    ],
    "Heating_Aging": [
        ("GPT 4o", 57, 51),
        ("Claude 3.7 Sonnet", 82, 75),
        ("Gemini 2.5 pro (preview)", 72, 73),
        ("Gemma3 27B", 95, 88)
    ],
    "Heating_Quenching": [
        ("GPT 4o", 50, 47),
        ("Claude 3.7 Sonnet", 60, 79),
        ("Gemini 2.5 pro (preview)", 70, 73),
        ("Gemma3 27B", 90, 94)
    ],
    "PressPress_Servo_Type": [
        ("GPT 4o", 59, 51),
        ("Claude 3.7 Sonnet", 66, 54),
        ("Gemini 2.5 pro (preview)", 73, 73),
        ("Gemma3 27B", 90, 85)
    ],
    "Rolling_hot": [
        ("GPT 4o", 63, 53),
        ("Claude 3.7 Sonnet", 76, 64),
        ("Gemini 2.5 pro (preview)", 82, 73),
        ("Gemma3 27B", 96, 80)
    ]
}

for task, results in tasks.items():
    for model, o3_score, claude4_score in results:
        data["Task"].append(task)
        data["Model"].append(model)
        data["o3"].append(o3_score)
        data["Claude 4"].append(claude4_score)

df = pd.DataFrame(data)

# === 2. HeatMap Function ===
def plot_llm_evaluation_heatmaps(df: pd.DataFrame, save_path: str = None):
    pivot_o3 = df.pivot(index="Model", columns="Task", values="o3")
    pivot_claude4 = df.pivot(index="Model", columns="Task", values="Claude 4")

    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    sns.heatmap(pivot_o3, annot=True, cmap="YlGnBu", fmt="d", cbar=True)
    plt.title("Evaluation by Opean AI o3")
    plt.xlabel("Task")
    plt.ylabel("Model")

    plt.subplot(1, 2, 2)
    sns.heatmap(pivot_claude4, annot=True, cmap="YlOrBr", fmt="d", cbar=True)
    plt.title("Evaluation by Claude 4 Opus")
    plt.xlabel("Task")
    plt.ylabel("Model")

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=600, bbox_inches="tight")
        print(f"✅ Success Save HeatMap Image: {save_path}")
    else:
        plt.show()

# === 3. Run ===
plot_llm_evaluation_heatmaps(df)


In [ ]:
plot_llm_evaluation_heatmaps(df, save_path="llm_heatmap_highres.png")